In [ ]:
%%file ./.jupyter/custom/custom.css
.left {
    width: 49%;
    float: left;
}

.right {
    width: 49%;
    float: right;
}

h1 {
    text-align: center;
}

h2 {
    text-align: center;
}

# Coordinates in SunPy

## Stuart Mumford


<a href="http://shef.ac.uk">
<img style='float: left; width: 30%; margin-top: 90px; height: 25%;' src='images/TUOS_Logo_CMYK_Keyline.svg'/>
</a>
<a href="https://aperiosoftware.com">
<img style='float: right; width: 30%; margin-top: 90px; height: 25%;' src='images/aperio_logo.svg'/>
</a>
<a href="http://sunpy.org">
<img style='float: right; width: 30%; margin-top: 90px; height: 25%; margin-right: 5%;' src='images/sunpy_logo.svg'/></a>

# Types of Coordinates

<div class='left'>
<h3> Pixel (data) Coordinates </h3>
<ul>
<li> The indexes of your array </li>
<li> WCS - World Coordinate System
<ul>
<li> Converts pixel coordinates to physical coordinates. </li>
<li> A common version is FITS-WCS </li>
</ul>
</ul>

</div>

<div class='right'>
<h3> Physical Coordinates </h3>
<ul>
<li> Represents a point in physical space. </li>
<li> Has some <i>"Reference System"</i>
    <ul>
    <li> An Origin </li>
    <li> Some reference points to align the frame with respect to other things. </li>
    </ul>
</li>
<li> Can be "projected" when spherical meaning $r\rightarrow \infty$.</li>
</ul>

</div>

# Physical Coordinate Frames

Defined in Thompson (2006):

* Helioprojective (Cartesian)
* Helioprojective (Radial) *
* Heliocentric
* Heliographic (Stonyhurst)
* Heliographic (Carrington)

<p style="font-size: small">
* Not in SunPy
<br/>
<br/>
1. Thompson, W. T. Coordinate systems for solar image data. Astronomy and Astrophysics 449, 791–803 (2006).
</p>

### Helioprojective Cartesian is not Cartesian
<div style="float:left; width:39%">
<p style="padding-top: 2em">
It's spherical.
</p>
</div>
<div style="float:left; width:59%">
<img src="images/coord_inset.png" width=100% />
</div>

# Transformations

<img width=200% src="images/sunpy_coordinates.svg"/>

# Physical Coordinates in SunPy

In [2]:
import astropy.units as u

from astropy.coordinates import SkyCoord
import sunpy.coordinates

In [3]:
sc = SkyCoord(100*u.arcsec, 0.1*u.deg, frame='helioprojective')
sc

<SkyCoord (Helioprojective: dateobs=None, rsun=695508.0 km, observer=<HeliographicStonyhurst Coordinate (dateobs=None): (lon, lat, radius) in (deg, deg, AU)
    ( 0.,  0.,  1.)>): (Tx, Ty) in arcsec
    ( 100.,  360.)>

In [5]:
sc.observer

<HeliographicStonyhurst Coordinate (dateobs=None): (lon, lat, radius) in (deg, deg, AU)
    ( 0.,  0.,  1.)>